In [5]:
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

df = pd.read_csv("data/players.csv")

In [10]:
# what is the cost of selecting a random subset of 15 players?
total_cost = 0
total_points_per_game = 0

for i in np.random.choice(df["id"], 15):
    cost = df[df["id"] == i ].now_cost.values[0] / 10
    points_per_game = df[df["id"] == i].points_per_game.values[0]
    position = df[df["id"] == i].position.values[0]
    print(f"Player {i}: Cost = ${cost}m, Position = {position}")
    total_cost+=cost
    total_points_per_game+=points_per_game

print(f"Total cost: ${total_cost}m")
print(f"Total points per game: {total_points_per_game}")

Player 71: Cost = $4.3m, Position = MID
Player 730: Cost = $4.5m, Position = MID
Player 499: Cost = $5.2m, Position = MID
Player 465: Cost = $4.4m, Position = DEF
Player 22: Cost = $5.3m, Position = MID
Player 109: Cost = $4.8m, Position = DEF
Player 86: Cost = $5.4m, Position = MID
Player 198: Cost = $4.8m, Position = DEF
Player 728: Cost = $4.0m, Position = DEF
Player 619: Cost = $4.0m, Position = DEF
Player 120: Cost = $4.4m, Position = MID
Player 679: Cost = $4.4m, Position = DEF
Player 193: Cost = $4.9m, Position = FWD
Player 214: Cost = $4.3m, Position = DEF
Player 251: Cost = $4.4m, Position = MID
Total cost: $69.1m
Total points per game: 18.1


#### Set the constraints
1. We need 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards
2. We cannot spend over $100m
3. We can pick no more than 3 players from the same team
4. Each player can only be picked once

Constraint #1

In [12]:
model = cp_model.CpModel()

# Define the number of players we want to select in each position
POSITION_MAP = {
    "Goalkeeper": {"code": "GKP", "count": 2},
    "Defender": {"code": "DEF", "count": 5},
    "Midfielder": {"code": "MID", "count": 5},
    "Forward": {"code": "FWD", "count": 3}
}

decision_variables = {}

In [13]:
# add constraints to each position
for position, details in POSITION_MAP.items():
    
    # find all eligible players, i.e. players who play the desired position
    players_in_position = list(df[df["position"] == details["code"]].id.values)

    # fetch the number of players we're allowed to pick in this position
    player_count = details["count"]

    # create decision variables for each of the players in that position
    player_variables = {i: model.NewBoolVar(f"player{i}") for i in players_in_position}
    decision_variables.update(player_variables) # add decision variable to global dict

    # add the constraint: per position, only allowed the specified count of players
    model.Add(sum(player_variables.values()) == player_count)

Constraint #2